### MET Database Cleaning

In [14]:
import pandas as pd
import numpy as np
from functools import reduce
import pycountry_convert as pc

In [2]:
#Load in sample for MET museum data
og_metsample_df = pd.read_csv('../data/raw/METsample.csv', on_bad_lines='skip')
og_metsample_df.columns

Index(['Object Number', 'Is Highlight', 'Is Timeline Work', 'Is Public Domain',
       'Object ID', 'Gallery Number', 'Department', 'AccessionYear',
       'Object Name', 'Title', 'Culture', 'Period', 'Dynasty', 'Reign',
       'Portfolio', 'Constituent ID', 'Artist Role', 'Artist Prefix',
       'Artist Display Name', 'Artist Display Bio', 'Artist Suffix',
       'Artist Alpha Sort', 'Artist Nationality', 'Artist Begin Date',
       'Artist End Date', 'Artist Gender', 'Artist ULAN URL',
       'Artist Wikidata URL', 'Object Date', 'Object Begin Date',
       'Object End Date', 'Medium', 'Dimensions', 'Credit Line',
       'Geography Type', 'City', 'State', 'County', 'Country', 'Region',
       'Subregion', 'Locale', 'Locus', 'Excavation', 'River', 'Classification',
       'Rights and Reproduction', 'Link Resource', 'Object Wikidata URL',
       'Metadata Date', 'Repository', 'Tags', 'Tags AAT URL',
       'Tags Wikidata URL', 'Random'],
      dtype='object')

In [3]:
#Narrow down dataframe to columns that could be useful for our analysis
met_df = og_metsample_df[['Title', 'Department', 'AccessionYear', 'Artist Display Name', 'Artist Nationality', 'Culture', 'Country', 'Region', 'Subregion',
                    'Classification', 'Tags', 'Object Wikidata URL']]
met_df.head()

,Title,Department,AccessionYear,Artist Display Name,Artist Nationality,Culture,Country,Region,Subregion,Classification,Tags,Object Wikidata URL
0,Calligraphic Exercise in Spanish,Drawings and Prints,2014.0,Anonymous,,NaN,NaN,NaN,NaN,Albums|Drawings|Ornament & Architecture,NaN,NaN
1,"Les Spectacles de Paris, ou, Calendrier histor...",The Libraries,NaN,Joseph de Laporte|Duchesne,|French,NaN,France,NaN,NaN,NaN,NaN,NaN
2,Set of Sword Fittings (Mitokoromono) with Two ...,Arms and Armor,1945.0,Gotō Jōshin,Japanese,Japanese,NaN,NaN,NaN,Sword Furniture,NaN,https://www.wikidata.org/wiki/Q116250603
3,Coat,Costume Institute,2005.0,Christian Lacroix|Christian Lacroix|Birger Chr...,French|French|Scandinavian,French,NaN,NaN,NaN,NaN,NaN,NaN
4,Churinga,"Arts of Africa, Oceania, and the Americas",1979.0,NaN,NaN,Mulga Downs Cave,Australia,Western Desert,NaN,Wood-Sculpture,NaN,NaN


In [4]:
met_df.columns

Index(['Title', 'Department', 'AccessionYear', 'Artist Display Name',
       'Artist Nationality', 'Culture', 'Country', 'Region', 'Subregion',
       'Classification', 'Tags', 'Object Wikidata URL'],
      dtype='object')

In [5]:
#Create a classification system that prioritizes information from country and culture columns over other columns in filling out 
#new country column, so we can extrapolate country from other columns if the information isn't in the original country column.
country_classification_columns = ('Country', 'Culture', 'Artist Nationality', 'Region', 'Subregion', 'Tags')
met_df['CombinedCountry'] = reduce(lambda x, y: x.combine_first(met_df[y]), country_classification_columns, met_df[country_classification_columns[0]])
met_df.head()                                                       

/var/folders/cy/f_z2j2b90rl7q8vz6jzfv__00000gn/T/ipykernel_54886/524570000.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  met_df['CombinedCountry'] = reduce(lambda x, y: x.combine_first(met_df[y]), country_classification_columns, met_df[country_classification_columns[0]])


,Title,Department,AccessionYear,Artist Display Name,Artist Nationality,Culture,Country,Region,Subregion,Classification,Tags,Object Wikidata URL,CombinedCountry
0,Calligraphic Exercise in Spanish,Drawings and Prints,2014.0,Anonymous,,NaN,NaN,NaN,NaN,Albums|Drawings|Ornament & Architecture,NaN,NaN,
1,"Les Spectacles de Paris, ou, Calendrier histor...",The Libraries,NaN,Joseph de Laporte|Duchesne,|French,NaN,France,NaN,NaN,NaN,NaN,NaN,France
2,Set of Sword Fittings (Mitokoromono) with Two ...,Arms and Armor,1945.0,Gotō Jōshin,Japanese,Japanese,NaN,NaN,NaN,Sword Furniture,NaN,https://www.wikidata.org/wiki/Q116250603,Japanese
3,Coat,Costume Institute,2005.0,Christian Lacroix|Christian Lacroix|Birger Chr...,French|French|Scandinavian,French,NaN,NaN,NaN,NaN,NaN,NaN,French
4,Churinga,"Arts of Africa, Oceania, and the Americas",1979.0,NaN,NaN,Mulga Downs Cave,Australia,Western Desert,NaN,Wood-Sculpture,NaN,NaN,Australia


In [6]:
#Check number of NaN values in CombinedCountry before mapping
met_df.isna().sum()

Title                    8884
Department                  0
AccessionYear            1219
Artist Display Name     62517
Artist Nationality      62517
Culture                 85752
Country                126584
Region                 140290
Subregion              143162
Classification          24330
Tags                    90312
Object Wikidata URL    128546
CombinedCountry          1718
dtype: int64

In [8]:
#Load in countries CSV and extract only the relevant columns
country_terms = pd.read_csv('../data/interim/CountriesCSV.csv')
country_terms = country_terms[['name.common', 'name.official', 'capital', 'altSpellings', 'demonyms.eng.m', 'demonyms.eng.f']]

#Drop rows that are causing issues with mapping due to same demonym as another row becaue the countries are not legitimately in dataframe
country_terms = country_terms[country_terms['name.common'] != 'British Indian Ocean Territory']
country_terms = country_terms[country_terms['name.common'] != 'United States Minor Outlying Islands']

#Adding autonomous regions as countries to avoid geographic confusion
new_rows = pd.DataFrame({'name.common': ['French Polynesia', 'Nepal', 'Tibet'], 'demonyms.eng.m': ['French Polynesian', 'Nepalese', 'Tibetan']})
country_terms = pd.concat([country_terms,new_rows], ignore_index=True)

#Changing NaN values to 'no data' so blank cells in column are not mapped to last country with a NaN value
country_terms.fillna('no data', inplace=True)
country_terms.tail(10)

,name.common,name.official,capital,altSpellings,demonyms.eng.m,demonyms.eng.f
244,Yemen,Republic of Yemen,Sana'a,"YE,Yemeni Republic,al-Jumhūriyyah al-Yamaniyyah",Yemeni,Yemeni
245,South Africa,Republic of South Africa,"Pretoria,Bloemfontein,Cape Town","ZA,RSA,Suid-Afrika,Republic of South Africa",South African,South African
246,Zambia,Republic of Zambia,Lusaka,"ZM,Republic of Zambia",Zambian,Zambian
247,Zimbabwe,Republic of Zimbabwe,Harare,"ZW,Republic of Zimbabwe",Zimbabwean,Zimbabwean
248,French Polynesia,no data,no data,no data,French Polynesian,no data
249,Nepal,no data,no data,no data,Nepalese,no data
250,Tibet,no data,no data,no data,Tibetan,no data
251,French Polynesia,no data,no data,no data,French Polynesian,no data
252,Nepal,no data,no data,no data,Nepalese,no data
253,Tibet,no data,no data,no data,Tibetan,no data


In [9]:
#Create dictionary from country_terms CSV file, making all alternative names map to common name
country_map = {}

for _, row in country_terms.iterrows():
    official_name = row['name.common']
    country_map[official_name] = official_name

    for col in country_terms.columns:
        if col != 'name.common':
            country_map[row[col]] = official_name


#Add alternate or ancient country names that are not in the original dictionary 
country_map.update({'Netherlandish': 'Netherlands', 'Bohemian': 'Czechia', 'Byzantine Egypt': 'Egypt', 'Flemish': 'Belgium',
                   'Scottish': 'United Kingdom', 'Welsh': 'United Kingdom', 'Korea' : 'Korea', 'England': 'United Kingdom',
                   'Scotland': 'United Kingdom', 'Wales': 'United Kingdom', 'Sumerian' : 'Iraq', 'Mesopotamia': 'Iraq',
                   'Etruscan': 'Italy', 'Minoan': 'Greece', 'West Bengal': 'India',
                    'Vendel': 'Sweden', 'Edomite': 'Jordan','Sino-Tibet': 'Tibet', 'Philippine': 'Philippines',
                    'North China': 'China', 'Alsatian': 'France', "Côte d'Ivoire": 'Ivory Coast', 'Malayan': 'Malaysia',
                   'Mycenaean': 'Greece', 'Republic of Congo': 'DR Congo','Hittite': 'Turkey',
                   'Sasanian': 'Iran', 'Sarawak': 'Malaysia', 'Aegean': 'Greece', 'South India': 'India',
                   'North German': 'Germany', 'Italic': 'Italy', 'Formosan': 'Taiwan',
                   'South Netherlands': 'Netherlands', 'Elamite': 'Iran', 'Javanese': 'Indonesia', 'North America': 'United States',
                    'Sicilian': 'Italy', 'Hattian': 'Turkey', 'America': 'United States',
                    'Macedonia': 'North Macedonia', 'Yi': 'China', 'Greek Islands': 'Greece', 'Greek (Attic)': 'Greece',
                    'Thailand (Ban Chiang)': 'Thailand', 'North French': 'France','Indian': 'India','Avar': 'Russia', 
                    'South Netherlandish': 'Netherlands', 'Alanic': 'Russia', 'Caroline Islands': 'Palau',
                   'Southern German': 'German', 'Native American': 'United States', 'Celtic': 'United Kingdom',
                   'Cycladic': 'Greece', 'Mangareva': 'French Polynesia', 'Burma': 'Myanmar', 'Franco-Netherlandish': 'France',
                   'the Republic of Congo': 'DR Congo', 'Persian': 'Iran', 'Persia': 'Iran', 'Rhodian': 'Greece',
                   'Indus': 'Pakistan', 'Nabataean': 'Jordan', 'Yortan': 'Turkey', 'North Spanish': 'Spain',
                   'Argentinian': 'Argentina', 'Swiss French': 'Switzerland', 'Campanian': 'Italy', 'The Netherlands': 'Netherlands',
                   'South Indian': 'India', 'USA': 'United States', 'Façon de Venise': 'Italy', 'Venetian': 'Italy',
                   'Lydian':'Turkey', 'Deccan': 'India', 'Sarmatian': 'Iran', 'Western Turkmenistan': 'Turkmenistan',
                   'Western Iran': 'Iran', 'Italic-Native': 'Italy', 'Villanovan': 'Italy', 'Baluchistan': 'Pakistan',
                   'South Italian': 'Italy', 'South German': 'Germany', 'Western Tibet': 'Tibet', 'west-central Turkey': 'Turkey',
                   'West Indian': 'India', 'Akkadian': 'Iraq', 'South Netherland': 'Netherlands', 'Chemehuevi': 'United States',
                   'Sumatran': 'Indonesia', 'Gemany': 'Germany', 'Egypt possibly': 'Egypt', 'Coptic': 'Egypt', 'Xiongnu': 'Mongolia',
                   'Phoenician': 'Lebanon', 'Mitanni': 'Turkey', 'North Netherlandish': 'Netherlands', 'Indigenous American': 'United States',
                   'Icelandic': 'Iceland', 'Nias': 'Indonesia', 'North Indian': 'India', 'Central India': 'India',
                   'Helladic': 'Greece','Marquesas Islands': 'French Polynesia', 'Nubia': 'Sudan', 'Dyak': 'Indonesia',
                   'North Netherland': 'Netherlands', 'Anatolia': 'Turkey', 'Silesian': 'Poland', 'Nias people': 'Indonesia',
                   'Greek Neolithic': 'Greece', 'Ubaid': 'Iraq', 'North Netherlands': 'Netherlands', 'Ottoman': 'Turkey',
                   'Anglo-Saxon': 'United Kingdom', 'Saxon': 'Germany', 'Phrygian': 'Turkey', 'Sassanian': 'Iran',
                   'Catalan': 'Spain', 'Chimú': 'Peru', 'Siberia': 'Russia', 'Democratic Republic of Congo': 'DR Congo',
                   'Urartian': 'Armenia', 'Neo-Sumerian': 'Iraq', 'New Guinea': 'Australia', 'Cretan': 'Greece',
                   'Babylonian': 'Iraq', 'Indonesia(': 'Indonesia', 'Russia Federation': 'Russia', 'Borneo': 'Indonesia',
                   'Minangkabau': 'Indonesia', 'Republic of Kiribati': 'Kiribati', 'Lapland': 'Finland', 'Vietnam(': 'Vietnam',
                   'Balinese': 'Indonesia', 'Egyptian possibly': 'Egypt', 'First Nations': 'Canada', 'South China': 'China',
                   'North Central Indian': 'India', 'Buganda': 'Uganda', 'Northern Italian': 'Italy', 'Madurese': 'Indonesia',
                   'Praenestine': 'Italy', 'Kirghiztan': 'Kyrgyzstan', 'Tuscany': 'Italy', 'North France': 'France',
                   'Canaanite': 'Palestine', 'North Italian': 'Italy', 'Western Australia': 'Australia', 'Façcon de Venise': 'Italy',
                   'Madura': 'Indonesia', 'Sardinian': 'Italy', 'Sienese': 'Italy', 'Venice': 'Italy', 'Sulawesi': 'Indonesia',
                   'Western India': 'India', 'Euboean': 'Greece', 'Congo': 'DR Congo', 'Southwestern German': 'Germany',
                   'Rumanian': 'Romania', 'Central Tibet': 'Tibet', 'Sinhala': 'Sri Lanka', 'Proto-Elamite': 'Iran',
                   'Ceylonese': 'Sri Lanka', 'Ojibwe': 'Canada', 'North India': 'India', 'Anatolian': 'Turkey', 'Kuna': 'Panama',
                   'Livonian': 'Latvia', 'Austia': 'Austria', 'British and American': 'United Kingdom', 'Batak': 'Indonesia',
                   'Locrian': 'Greece', 'Argentinean': 'Argentina', 'North Central Thailand': 'Thailand', 'northern France': 'France',
                   'Northwestern Iran': 'Iran', 'Flanders': 'Belgium', 'Crimean': 'Ukraine', 'Bornean': 'Indonesia',
                   'Leipzig': 'Germany', 'iran': 'Iran', 'Alemannic': 'Switzerland', 'Friesland': 'Netherlands', 
                   'Southwestern Iran': 'Iran', 'Diné': 'United States', 'German(': 'Germany', 'Southwest Italian': 'Italy',
                   'Greenlandish': 'Denmark', "Cote d'Ivoire": 'Ivory Coast', 'South French': 'France', 'Tewa': 'United States',
                   'Southern French': 'France', 'Central Italian': 'Italy', 'Central Turkey': 'Turkey', 'English': 'United Kingdom',
                   'Volga region': 'Russia', 'Central Italy': 'Italy', 'West Central Turkey': 'Turkey', 'Bohemia': 'Czechia',
                   'southern England': 'United Kingdom', 'Dimini culture': 'Greece', 'the Netherlands': 'Netherlands',
                   'Slovakian': 'Slovakia', 'Native Italic': 'Italy', 'Façon de venise': 'Italy', 'Antwerp': 'Belgium',
                   'Peninsular Thailand': 'Thailand', 'Sesklo culture': 'Greece', 'North Swiss': 'Switzerland',
                   'Siberian': 'Russia', 'Apuilan': 'Italy', 'Republic of  Cameroon': 'Cameroon', 'Central Indian': 'India',
                   'Corsican': 'France', 'Southern Netherlandish': 'Netherlands', 'Central Iran': 'Iran', 'Hacilar': 'Turkey',
                   'Carthaginian': 'Tunisia', 'Circassian': 'Russia', 'Isin-Larsa': 'Iraq', 'Republic of Timor-Leste': 'Timor-Leste',
                   'UK': 'United Kingdom', 'Central Anatolia': 'Turkey', 'Native Italian': 'Italy', 'Central German': 'Germany',
                   'Anvers': 'Belgium', 'German': 'Germany'})


In [10]:
#Clean up white space and eliminate punctuation and prefixes from CombinedCountry column
met_df['CombinedCountry'] = met_df['CombinedCountry'].str.strip()
met_df['CombinedCountry'] = met_df['CombinedCountry'].str.split(r'[,/.:;?]| or| \(').str[0]
met_df['CombinedCountry'] = met_df['CombinedCountry'].str.lstrip('|')
met_df['CombinedCountry'] = met_df['CombinedCountry'].str.split('|').str[0]

met_df['CombinedCountry'] = met_df['CombinedCountry'].str.replace(r'^(northern|southern|eastern|east|probably|northeastern|northwest|northwestern|northeast|southeastern)\s+','', case=False, regex=True)
met_df['CombinedCountry'] = met_df['CombinedCountry'].str.replace(r'^(possibly|Byzantine|present-day|Colonial|north-central|)\s+','', case=False, regex=True)

#Map CombinedCountry column to the country names dictionary
met_df['MappedCountry'] = met_df['CombinedCountry'].map(country_map)

/var/folders/cy/f_z2j2b90rl7q8vz6jzfv__00000gn/T/ipykernel_54886/3041950666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  met_df['CombinedCountry'] = met_df['CombinedCountry'].str.strip()
/var/folders/cy/f_z2j2b90rl7q8vz6jzfv__00000gn/T/ipykernel_54886/3041950666.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  met_df['CombinedCountry'] = met_df['CombinedCountry'].str.split(r'[,/.:;?]| or| \(').str[0]
/var/folders/cy/f_z2j2b90rl7q8vz6jzfv__00000gn/T/ipykernel_54886/3041950666.py:4: SettingWithCopyW

In [11]:
#Narrow down dataframe to columns necessary for analysis and visualization
met_df = met_df[['Title', 'AccessionYear', 'Department', 'Country', 'Culture', 'Artist Nationality', 'Region','CombinedCountry', 'MappedCountry']]

#Check how many rows are still missing country information
met_df.isna().sum()

Title                   8884
AccessionYear           1219
Department                 0
Country               126584
Culture                85752
Artist Nationality     62517
Region                140290
CombinedCountry         1718
MappedCountry          12952
dtype: int64

In [ ]:
#Convert cleaned MET dataframe to downloadable CSV
from IPython.display import FileLink
CleanMETData = 'CleanMETData.csv'
# met_df.to_csv(CleanMETData)
FileLink(CleanMETData)

/Users/mollystark/Desktop/michigan/milestone 1/SIADS-Milestone-1/notebooks/CleanMETData.csv

In [13]:
#Convert cleaned country dataframe to downloadable CSV
from IPython.display import FileLink
CountriesCSV = 'CountriesCSV.csv'
country_terms.to_csv(CountriesCSV)
FileLink(CountriesCSV)

/home/jovyan/work/CountriesCSV.csv

In [14]:
#Gather values that were not mapped to a country to see if they can be added to dictionary manually
deleted_vals = met_df[met_df['MappedCountry'].isna()]['CombinedCountry']
deleted_vals

#Convert deleted values dataframe to downloadable CSV for easy reading
from IPython.display import FileLink
del_vals = 'del_vals.csv'
deleted_vals.to_csv(del_vals)
FileLink(del_vals)

/home/jovyan/work/del_vals.csv

Adding in continent data using MappedCountry column

In [15]:
country_and_continent = []

def country_to_continent(country_name):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name
    except KeyError:
        return None

countries_list = list(set(met_df['MappedCountry'].dropna()))
for i in range(len(countries_list)):
    if country_to_continent(countries_list[i]) is not None:
        country_and_continent.append((countries_list[i],country_to_continent(countries_list[i])))

country_and_continent.append(('Timor-Leste', 'Asia'))
country_and_continent.append(('DR Congo', 'Africa'))
country_and_continent.append(('Tibet', 'Asia'))

In [16]:
country_and_continent_df = pd.DataFrame(country_and_continent, columns =['MappedCountry', 'continent']).dropna()
met_continent = pd.merge(met_df, country_and_continent_df, on='MappedCountry', how='left')

Changing column names for ease of use & readability

In [17]:
met_continent.columns

Index(['Title', 'AccessionYear', 'Department', 'Country', 'Culture',
       'Artist Nationality', 'Region', 'CombinedCountry', 'MappedCountry',
       'continent'],
      dtype='object')

In [19]:
met = met_continent.rename(columns={'Title': 'title',
                    'AccessionYear':'accession_year',
                    'Department':'department',
                    'Country':'listed_country',
                    'Culture':'culture',
                    'Artist Nationality':'artist_nationality',
                    'Region':'region',
                    'CombinedCountry':'combined_country',
                    'MappedCountry': 'country',
                    })

# met = met.drop(labels=['Unnamed: 0.1', 'Unnamed: 0'],axis=1)

In [20]:
met.to_csv('../data/processed/met_data.csv')

### 

### 